
(sec:mol-hessian)=
# Molecular Hessians

This section deals with the calculation of second derivatives of the electronic energy with respect to nuclear coordinates.
While the numerical calculation of the Hessian based on an analytical gradient follows the same procedure as for the 
[numerical gradient](sec:numerical-gradients),
the analytical calculation is in general much more involved than for first derivatives.
After some special aspects of the numerical calculation of second derivatives,
the calculation of the analytic Hessian at the Hartree--Fock (HF) level of theory is described as an example.

## Numerical Hessian

If the analytical gradient is not available, it is more efficient to resort to an equation for the second derivative
that depends on the energy $E$ only, rather than employing one of the equations [here](sec:numerical-gradients)
with the energy itself replaced by its numerical gradient.
Assuming the energy depends on (at least) two distinct nuclear coordinates $\xi$ and $\chi$, the second derivative of $E$
can be calculated as

$$
\frac{\mathrm{d}^2 E}{\mathrm{d} \chi \mathrm{d} \xi} \approx \frac{E(\xi + \xi_0, \chi + \chi_0) - E(\xi + \xi_0, \chi) - E(\xi, \chi + \chi_0) + 2 E(\xi, \chi) - E(\xi - \xi_0, \chi) - E(\xi, \chi - \chi_0) + E(\xi - \xi_0, \chi - \chi_0)}{2 \xi_0 \chi_0} \, ,
$$

where $\chi_0$ and $\xi_0$ are some small changes in the respective variable.
For the case that the deritative is taken with respect to the same nuclear coordinate, the formula simplifies to

$$
\frac{\mathrm{d}^2 E}{\mathrm{d} \xi^2} \approx \frac{E(\xi + \xi_0, \chi) - 2 E(\xi, \chi) + E(\xi - \xi_0, \chi)}{\xi_0^2} \, .
$$

## Analytical Hessian in Hartree--Fock


The [analytic gradient of the HF energy](sec:hf-gradients) $E_{\text{HF}}$ with respect to a nuclear coordinate $\xi$,
which was [previously given](eq:HF_grad_final) in MO basis, can equivalently be written in AO basis as {cite}`Pople1979`
(eq:HF_gradient_in_ao)=
```{math}
%:label: eq:HF_gradient_in_ao
  \frac{\mathrm{d} E_{\text{HF}}}{\mathrm{d} \xi} = \sum_{\mu \nu} P_{\mu \nu} h_{\mu \nu}^{\xi}
		+ \frac12 \sum_{\mu \nu \kappa \lambda} P_{\mu \nu} P_{\kappa \lambda} \langle \mu \kappa || \nu \lambda \rangle^\xi
		+ \sum_{\mu \nu} \omega_{\mu \nu} S_{\mu \nu}^{\xi} + \frac{\mathrm{d} V_{nn}}{\mathrm{d} \xi} \, ,
```
where the HF density matrix $\mathbf{P}$ and energy-weighted density matrix $\boldsymbol{\omega}$ in a real AO basis are defined as
```{math}
  P_{\mu \nu} &= \sum_{i} C_{\mu i} C_{\nu i} \\
  \omega_{\mu \nu} &= - \sum_{i} \varepsilon_i C_{\mu i} C_{\nu i}
```

Straightforward differentiation of the [above equation](eq:HF_gradient_in_ao) with respect to another nuclear coordinate $\chi$ gives second derivatives of the HF energy:
```{math}
:label: eq:HF_Hessian_Pople
  \frac{\mathrm{d}^2 E_{\text{HF}}}{\mathrm{d} \chi \mathrm{d} \xi} &= \sum_{\mu \nu} P_{\mu \nu} h_{\mu \nu}^{\xi \chi} + \frac12 \sum_{\mu \nu \lambda \sigma} P_{\mu \nu} P_{\lambda \sigma} \langle \mu \lambda || \nu \sigma \rangle^{\xi \chi} + \sum_{\mu \nu} \omega_{\mu \nu} S_{\mu \nu}^{\xi \chi} + \frac{\mathrm{d}^2 V_{nn}}{\mathrm{d} \chi \mathrm{d} \xi} \\
  &+ \sum_{\mu \nu} \frac{\mathrm{d} P_{\mu \nu}}{\mathrm{d} \chi} h_{\mu \nu}^{\xi} + \sum_{\mu \nu \lambda \sigma} \frac{\mathrm{d} P_{\mu \nu}}{\mathrm{d} \chi} P_{\lambda \sigma} \langle \mu \lambda || \nu \sigma \rangle^{\xi} + \sum_{\mu \nu} \frac{\mathrm{d} \omega_{\mu \nu}}{\mathrm{d} \chi} S_{\mu \nu}^{\xi}
```

Now, the calculation of the perturbed density, which boils down to the calculation
of the perturbed MO coefficients $C_{\mu p}^{\chi}$, can no longer be avoided {cite}`Pople1979`.
How this is achieved in practice will be described in the following.




(sec:cphf)=
## Coupled-perturbed Hartree--Fock

The starting point are the HF equations in their basis-set form,
```{math}
  \sum_{\nu} F_{\mu \nu} (\mathbf{C}) C_{\nu i} = \varepsilon_i \sum_{\nu} S_{\mu \nu} C_{\nu i} \, ,
```
which can be differentiated and rearranged, such that one obtains {cite}`Neese2009`
(eq:derivative_hf_equations)=
```{math}
%:label: eq:derivative_hf_equations
  \sum_{\nu} \Big( \frac{\mathrm{d} F_{\mu \nu} (\mathbf{C})}{\mathrm{d} \chi} C_{\nu i}
  + ( F_{\mu \nu} (\mathbf{C}) - \varepsilon_i S_{\mu \nu} ) \frac{\mathrm{d} C_{\nu i}}{\mathrm{d} \chi} \Big)
  = \frac{\mathrm{d} \varepsilon_i}{\mathrm{d} \chi} \sum_{\nu} S_{\mu \nu} C_{\nu i}
  + \varepsilon_i \sum_{\nu} \frac{\mathrm{d} S_{\mu \nu}}{\mathrm{d} \chi} C_{\nu i} \, .
```

Next, an ansatz for the perturbed MO coefficients $C_{\mu p}^{\chi} = \frac{\mathrm{d} C_{\mu p}}{\mathrm{d} \chi}$ needs to be made.
Since the unperturbed MOs span the same space as the AOs, but form an orthonormal set,
the perturbed MO coefficients are expanded in the basis of the unperturbed ones {cite}`Pople1979`,
(eq:perturbed_mo_coefficients)=
```{math}
%:label: eq:perturbed_mo_coefficients
  C_{\mu p}^{\chi} &= \sum_{q} C_{\mu q} U_{q p}^{\chi} \, , \\
```
where the matrix $\mathbf{U}^\chi$ contains the unknown expansion coefficients.
By taking the derivative of the orthonormality condition $\mathbf{C}^\text{T} \mathbf{SC} = \mathbf{1}$,
or in subscript notation {cite}`Neese2009`,
```{math}
  \sum_{\mu \nu} C_{\mu p} S_{\mu \nu} C_{\nu q} = \delta_{pq} \, ,
```
it can be shown that the occupied-occupied block of $\mathbf{U}^\chi$
is proportional to the partial derivative of the overlap matrix,
\begin{equation*}
  U_{ij}^{\chi} = - \frac12 S_{ij}^{(\chi)} \, .
\end{equation*}

The virtual-virtual block is not required, and hence only the occupied-virtual block $U_{ai}^\chi$ needs to be determined.
The derivative of the Fock matrix $F_{\mu \nu}^{\chi} = \frac{\mathrm{d} F_{\mu \nu} (\mathbf{C})}{\mathrm{d} \chi}$ is given by
(eq:derivative_fock_matrix)=
```{math}
%:label: eq:derivative_fock_matrix
  F_{\mu \nu}^{\chi} = h_{\mu \nu}^{\chi}
  + \sum_{\kappa \lambda} P_{\kappa \lambda} \langle \mu \kappa || \nu \lambda \rangle^{\chi}
  + \sum_{qi} \sum_{\kappa \lambda} (U_{qi}^{\chi} C_{\kappa q} C_{\lambda i}
  + U_{qi}^{\chi} C_{\kappa i} C_{\lambda q} ) \langle \mu \kappa || \nu \lambda \rangle \, ,
```
where it can be seen that the derivative of the Fock matrix also depends on the unknowns $\mathbf{U}^{\chi}$.
The **coupled-perturbed Hartree--Fock** (CPHF) equations are obtained by multiplying the [derivative of the HF equations](eq:derivative_hf_equations)
from the left by $C_{\mu a}$ and summing over all $\mu$ {cite}`Neese2009`.
The CPHF equations, whose solution yields the $U_{ai}^{\chi}$, can be written as {cite}`Deglmann2002`
(eq:cphf-equations)=
\begin{align*}
  (\varepsilon_i - \varepsilon_a) U_{ai}^{\chi} - 2 G_{ai}[U_{bj}^{\chi}] = R_{ai}^{\chi} \, ,
\end{align*}
where the right-hand side (RHS), which depends on the perturbation $\chi$, is given by
(eq:cphf-rhs)=
\begin{equation*}
  R_{ai}^{\chi} = F_{ai}^{(\chi)} - \varepsilon_{i} S_{ai}^{(\chi)} - G_{ai}[S_{jk}^{(\chi)}] \, ,
\end{equation*}
and the matrices $\mathbf{G}[M_{rs}^{\chi}]$, for arbitrary $M_{rs}^{\chi}$, are defined as {cite}`Deglmann2002`
\begin{equation*}
  G_{pq}[M_{rs}^{\chi}] = \sum_{\mu \nu \kappa \lambda} C_{\mu p} C_{\nu q} \langle \mu \kappa || \nu \lambda \rangle \sum_{rs} C_{\kappa r} C_{\lambda s} M_{rs}^{\chi} \, .
\end{equation*}

It should be noted that in contrast to the first derivative of the MP2 or CIS energy, where only a single orbital-response equation had to be solved,
a response equation needs to be solved for every perturbation $\chi$, i.e., the $3N$ Cartesian nuclear coordinates of a molecule containing $N$ atoms,
to obtain the second derivative of the HF energy.

Having solved for the CPHF coefficients $\mathbf{U}^{\chi}$, the perturbed density matrix can be calculated by using the chosen [ansatz](eq:perturbed_mo_coefficients) as
(eq:perturbed_density)=
```{math}
%:label: eq:perturbed_density
  \frac{\mathrm{d} P_{\mu \nu}}{\mathrm{d} \chi} &= \sum_{i} \frac{\mathrm{d}}{\mathrm{d} \chi} (C_{\mu i} C_{\nu i}) = \sum_{i} ( C_{\mu i}^\chi C_{\nu i} + C_{\mu i} C_{\nu i}^\chi ) \\
  &= - \frac12 \sum_{ij} (  C_{\mu i} C_{\nu j} S_{ij}^{(\chi)} + C_{\mu j} C_{\nu i} S_{ij}^{(\chi)} ) + \sum_{ia} ( C_{\mu a} C_{\nu i} + C_{\mu i} C_{\nu a} ) U_{ai}^{\chi} \\
```


As an alternative to [this equation](eq:HF_Hessian_Pople), the second derivatives of the HF energy can then be written
in terms of the RHS $R_{ai}^{\chi}$, the CPHF coefficients $U_{ai}^{\xi}$, and partial derivatives of the Fock and overlap matrices as {cite}`Deglmann2002`
```{math}
%:label: eq:HF_Hessian_Furche
  \frac{\mathrm{d}^2 E_{\text{HF}}}{\mathrm{d} \chi \mathrm{d} \xi} &= \sum_{\mu \nu} P_{\mu \nu} h_{\mu \nu}^{\chi \xi}
	+ \frac12 \sum_{\mu \nu \lambda \sigma} P_{\mu \nu} P_{\lambda \sigma} \langle \mu \lambda || \nu \sigma \rangle^{\xi \chi}
	+ \sum_{\mu \nu} \omega_{\mu \nu} S_{\mu \nu}^{\chi \xi} + \frac{\mathrm{d}^2 V_{nn}}{\mathrm{d} \chi \mathrm{d} \xi} \\
  &+ 2 \sum_{ia} R_{ai}^{\chi} U_{ai}^{\xi} - \sum_{ij} \Big( F_{ij}^{(\chi)} S_{ij}^{(\xi)} + F_{ij}^{(\xi)} S_{ij}^{(\chi)}
	- 2 \varepsilon_i S_{ij}^{(\chi)} S_{ij}^{(\xi)} - 2 G_{ij}[S_{kl}^{(\chi)}] S_{ij}^{(\xi)} \Big) \, ,
```
where the explicit construction of the perturbed $\mathbf{P}$ and $\boldsymbol{\omega}$ matrices is avoided.

(sec:second-order-prop)=
## Second-order properties

Analogous to the case of [first-order properties](sec:first-order-prop),
also other time-independent properties molecular properties can be calculated as second derivatives
of the energy $E$. As an example, consider the static dipole polarizability $\boldsymbol{\alpha}$,
which corresponds to the derivative of the dipole moment $\boldsymbol{\mu}$
with respect to an external electric field $\boldsymbol{\mathcal{F}}$,

$$
\boldsymbol{\alpha} = \frac{\mathrm{d} \boldsymbol{\mu}}{\mathrm{d} \boldsymbol{\mathcal{F}}}
\approx \frac{\boldsymbol{\mu}(\boldsymbol{\mathcal{F}}) - \boldsymbol{\mu}(-\boldsymbol{\mathcal{F}})}{2 \boldsymbol{\mathcal{F}}} \, ,
$$

which can be obtained numerically by calculating the (analytical) dipole moment in presence of
an external field in positive and negative direction.
Since the dipole moment is already the first derivative of the energy $E$ with respect to the field,
the polarizability is actually a second derivative,

$$
\boldsymbol{\alpha} = \frac{\mathrm{d}^2 E}{\mathrm{d} \boldsymbol{\mathcal{F}}^2} \, ,
$$

and as such can be referred to as a **second-order property**.
The analytic calculation at the HF level procedes analogously to the nuclear Hessian,
meaning that the [CPHF equations](eq:cphf-equations) need to be solved for the unknowns
$U_{ia}^{\mathcal{F}}$. However, since the basis functions do not depend on the electric field,
the RHS is in this case simply given by a component $\{ x,y,z \}$ of the dipole integrals, $R_{ia}^{\mathcal{F}} = \mu_{ia}$.
As a matter of fact, this equation is then equivalent to the linear response equation with $\omega = 0$.
Having solved for the unknown CPHF coefficients, the perturbed density is obtained from

$$
P_{\lambda \kappa}^{\mathcal{F}} = \frac{\mathrm{d} P_{\lambda \kappa}}{\mathrm{d} \mathcal{F}} = \sum_{ia} \big( C_{\lambda i} C_{\kappa a} + C_{\lambda a} C_{\kappa i} \big) U_{ia}^{\mathcal{F}}
$$

and the polarizability components can be calculated as

$$
\alpha = \sum_{\kappa \lambda} P_{\lambda \kappa}^{\mathcal{F}} \, \mu_{\kappa \lambda}
$$

for each component of the perturbed density and dipole integrals.
Another example of a second-order property is the derivative of the dipole moment with respect to
nuclear coordinates, which is needed for IR intensities.
This can be seen as a mixed second energy derivative (once with respect to nuclear coordinates, once with respect to an electric field), and is discussed in more detail [here](dipole_mom_gradient:label).